In [2]:
import numpy as np

In [181]:
def obj(x):
    return 10*np.sin(x[0,:])**2 + 10*np.cos(x[2,:])**2 + (x[1,:]- 7)**2 + (x[3,:]- 2)**2

In [182]:
def get_best_values(metric_current, positions, metric_best_global, metric_best_locals, i):
    best_local_index = metric_current < metric_best_locals
    metric_best_locals = metric_current[best_local_index]
    local_best_position = positions[:, best_local_index[0, :], i]
    min_index = metric_current.argmin(axis=1)
    if metric_current[0, min_index] < metric_best_global:
        metric_best_global = metric_current[0, min_index]
        global_best_position = positions[:, min_index, i]
    return  global_best_position, local_best_position

In [183]:
def update_continuous_positions(continuous_mask, velocities, inertia_weight,cognitive_weight,num_particles,social_weight,positions,i):
    inertial_velocity = inertia_weight * velocities[continuous_mask, :, i:i + 1]
    cognitive_velocity = cognitive_weight * velocities[:, :, i:i + 1] * np.random.uniform(low=0, high=1, size=(continuous_mask.sum(), num_particles, 1))
    social_velocity = social_weight * velocities[:, :, i:i+1] * np.random.uniform(low=0, high=1, size=(continuous_mask.sum(), num_particles, 1))
    velocities[:, :, i + 1:i + 2] = inertial_velocity + cognitive_velocity + social_velocity
    positions[:, :, i + 1:i + 2] = positions[:, :, i:i + 1] + velocities[:, :, i:i + 1]
    return positions, velocities

In [ ]:
def int2binary(x)->np.ndarray:
    binaries = np.zeros((2,200,16))
    msk = x>0
    i = 0
    while np.any(msk):
        rem = x%2
        binaries[msk,i] = rem[msk]
        x = x//2
        msk = x>0
        i += 1
    return binaries

def binary2int(x)->np.ndarray:
    return (x*np.tile(1<<np.arange(16),reps=(1,200,1))).sum(axis=2).astype(int)


def binary_crossover(local, best, current, n_dis):
    contribution_global = 0.5
    contribution_local = 0.2
    m = 16
    n_global = np.floor(contribution_global * m).astype(int)
    n_local = np.floor(contribution_local * m).astype(int)
    new_ = np.zeros((n_dis,200, m),dtype=int)
    indexes = np.random.choice(range(m), size=(m,), replace=False)
    new_[:,:,indexes[:n_global]] = np.tile(best[:,indexes[:n_global]],reps=(1, 200,1))
    new_[:,:,indexes[n_global:n_global + n_local]] = local[:,:,indexes[n_global:n_global + n_local]]
    new_[:,:,indexes[n_global + n_local:]] = current[:,:, indexes[n_global + n_local:]]
    return binary2int(new_)

In [288]:
def saturator(x,lb,ub):
    msk_lb, msk_ub = x <= lb, x >= ub
    x[msk_lb], x[msk_ub] = lb[msk_lb], ub[msk_ub]
    return x

In [ ]:
def get_best_values(positions,continuous_mask, metric_current,local_best_position, metric_best_locals,metric_best_global, i):
    best_local_index = metric_current < metric_best_locals
    metric_best_locals = metric_current[best_local_index]
    local_best_position[:,best_local_index[0, :]] = positions[:, best_local_index[0, :], i]
    min_index = metric_current.argmin(axis=1)
    if metric_current[0, min_index] < metric_best_global:
        metric_best_global = metric_current[0, min_index]
        global_best_position = positions[:, min_index, i]
    return global_best_position, metric_best_global, metric_best_locals, local_best_position


def update_cont_values(n_con, global_best_position, local_best_position, positions,inertia_weight,cognitive_weight, social_weight, dim, num_particles, velocities,continuous_mask,i):
    inertial_velocity = inertia_weight * velocities[:,:, i:i + 1]
    cognitive_velocity = cognitive_weight * np.random.uniform(low=0, high=1, size=(n_con, num_particles, 1)) * (global_best_position[continuous_mask,:] - positions[continuous_mask, :, i:i + 1])
    social_velocity = social_weight * np.random.uniform(low=0, high=1, size=(n_con, num_particles, 1)) * (local_best_position[continuous_mask,:] - positions[continuous_mask, :, i:i + 1])
    velocities[:, :, i + 1:i + 2] = inertial_velocity + cognitive_velocity + social_velocity
    positions[continuous_mask, :, i + 1:i + 2] = positions[continuous_mask, :, i:i + 1] + velocities[:, :, i:i + 1]
    return positions

def update_disc_values(n_dis, global_best_position, local_best_position, positions, discrete_mask):
    binary_current = int2binary(positions[discrete_mask, :])
    binray_global = int2binary(current_position)  # to binary
    binary_local =  int2binary(local_best_position[discrete_mask,:])  # to binary
    

    return



def opt(fcn, num_particles, max_iter, inertia_weight,
        cognitive_weight, social_weight, V_max, lb, ub, int_idx):
    
    dim = len(lb)
    continuous_mask = np.ones((dim,), dtype=bool)
    continuous_mask[int_idx] = False
    discrete_mask = ~continuous_mask
    n_dis = discrete_mask.sum()
    n_con = discrete_mask.sum()
    lb = np.tile(A=np.array(lb).reshape((-1, 1)), reps=(1, num_particles))
    ub = np.tile(A=np.array(ub).reshape((-1, 1)), reps=(1, num_particles))

    metric_best_locals = np.ones((num_particles,)) * np.inf  # (particles)
    param_best_locals = np.ones((dim, num_particles)) * np.inf  # (dim x particles)

    metric_track = np.ones((num_particles, max_iter)) * np.inf
    positions = np.zeros((dim, num_particles, max_iter))    
    positions[:, :, 0:1] = (np.random.uniform(low=0, high=1, size=(dim, num_particles)) * (
                ub - lb) + lb)[:, :, np.newaxis]
    
    positions[discrete_mask, :, 0:1] = np.round(positions[discrete_mask, :, 0:1])
    positions = np.minimum(np.maximum(positions[:, :, 0], lb), ub)[:, :, np.newaxis]
    velocities = np.zeros((n_con,num_particles))
    
    metric_current = np.ones((1, num_particles)) * np.inf
    # best obtained metric
    metric_best_global = np.inf

    # initializing best values of parameters (positions)
    global_best_position = np.tile(A=positions[:, 0, 0].reshape((-1, 1)), reps=(1, num_particles))
    local_best_position = positions[:, :, 0]

    for i in range(max_iter - 1):
        metric_current = fcn(positions[:,:,i])
        global_best_position, metric_best_global, metric_best_locals, local_best_position = get_best_values(positions,continuous_mask, metric_current,local_best_position, metric_best_locals,metric_best_global, i)
        positions = update_cont_values(n_con, global_best_position, local_best_position, positions,inertia_weight,cognitive_weight, social_weight, dim, num_particles, velocities,continuous_mask,i)

        




In [214]:
current  = 2
best = 45
local = 33
def bit2int(m):
    return (np.flip(1<<np.arange(16))*m).sum()

def int2bit(q):
    return np.fromiter(np.binary_repr(q,width=16), dtype=int)

def new_Crossover(local,best,current):
    new = np.ones((16,))
    
    
    return a, b

s1 = int2bit(current)
s2 = int2bit(best)
s3 = int2bit(local)

In [232]:
np.unpackbits(np.array([2,3,6], dtype=np.uint8)).reshape(-1, 8)


TypeError: Expected an input array of unsigned byte data type

In [44]:
d = np.array([1,2,3,4,5])
m = 8
def conv_(d):
    return (((d[:,None] & ())) > 0).astype(int)

In [266]:

f1

(4, 20, 16)

In [242]:
d = np.array([1,2,3,4,5])

In [244]:
d%2

array([1, 0, 1, 0, 1], dtype=int32)

In [282]:
x= np.random.randint(low=0,high=15,size=(4,20))
x

array([[13,  4,  6, 10,  8,  3,  5, 13, 13,  0, 14, 10,  4,  9,  7,  4,
        11,  5, 13,  6],
       [ 5,  4, 13, 11,  5, 11,  9, 11, 11,  9, 10,  6,  7, 12, 11,  1,
         2, 14,  9,  0],
       [ 1, 13,  5,  5,  8, 12,  4,  3,  8, 10,  9, 14,  8, 14, 14, 11,
         0,  0, 13,  6],
       [ 9,  1,  0,  6,  2,  3, 11,  0, 13, 13,  2,  8, 11,  2,  2,  4,
        12,  6, 10,  3]])

In [286]:
rr = int2binary(x)


array([[13,  4,  6, 10,  8,  3,  5, 13, 13,  0, 14, 10,  4,  9,  7,  4,
        11,  5, 13,  6],
       [ 5,  4, 13, 11,  5, 11,  9, 11, 11,  9, 10,  6,  7, 12, 11,  1,
         2, 14,  9,  0],
       [ 1, 13,  5,  5,  8, 12,  4,  3,  8, 10,  9, 14,  8, 14, 14, 11,
         0,  0, 13,  6],
       [ 9,  1,  0,  6,  2,  3, 11,  0, 13, 13,  2,  8, 11,  2,  2,  4,
        12,  6, 10,  3]])

array([[ 1., 11.,  6.,  2.,  8.,  5.,  5.,  7., 12., 12., 14., 10., 12.,
         5., 10.,  1.,  5.,  4.,  9.,  3.],
       [10., 13.,  7.,  6., 12.,  3., 13.,  2.,  0., 14.,  9.,  7.,  4.,
         6., 12.,  2.,  2.,  1.,  0.,  4.],
       [ 7.,  3., 11., 11., 10.,  1., 12.,  0., 13.,  0.,  0.,  2.,  0.,
         4.,  5.,  8., 13.,  3., 14., 12.],
       [ 0.,  9.,  0.,  4., 13.,  4.,  6.,  9., 13.,  4.,  6.,  8.,  1.,
         1.,  1.,  5.,  1., 13., 14.,  2.]])

In [274]:
f1[0,1,:]

array([    1,     2,     4,     8,    16,    32,    64,   128,   256,
         512,  1024,  2048,  4096,  8192, 16384, 32768], dtype=int32)

In [251]:
binaries = np.zeros((3,4,16))
x = np.ones((3,4))
msk = x<0
msk[1,1] = True



array([[False, False, False, False],
       [False,  True, False, False],
       [False, False, False, False]])

In [254]:
binaries[msk,0]=1
binaries[:,:,0]

array([[0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.]])

In [192]:
np.random.randint(low=0,high=16,size=(16,))
np.random.choice([0,1,2],size=(16,),replace=True)


array([0, 2, 0, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1, 2, 0])

In [209]:
indexes = np.random.choice(range(16),size=(16,),replace=False)


In [213]:
indexes[:n_g], indexes[n_g:n_g+n_l], indexes[n_g+n_l:]

(array([15, 12,  3,  9, 14,  7,  1,  0]),
 array([ 4, 13, 11]),
 array([ 2, 10,  6,  8,  5]))

[8, 3, 5]


In [170]:
import random
def crossover(a, b):
    n = 16
    crossover_point = random.randint(1, n - 1)
    # a[crossover_point:], b[crossover_point:] = b[crossover_point:], a[crossover_point:]
    a[::2],b[::2] = b[::2],a[::2]
    return a, b
#--------------------------------------------------------------------------------
def crossover_every_n(a,b,n):
    a[::n],b[::n] = b[::n],a[::n]
    return a, b

In [171]:
def bit2int(m):
    return (np.flip(1<<np.arange(16))*m).sum()
def int2bit(q):
    return np.fromiter(np.binary_repr(q,width=16), dtype=int)

In [174]:
r1 = 150
r2 = 17
s1 = int2bit(r1)
s2 = int2bit(r2)
s1,s2


(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]))

In [176]:
s1_1, s2_2 = crossover_every_n(s1, s2,3)

In [180]:
def obj(x):
    return x[0,:]**2 + (1-x[1,:]**2)

In [178]:

for i in range(200):

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [179]:
bit2int(s1_1),bit2int(s2_2)

(151, 17)

In [161]:
cc = 1<<np.arange(16)

In [162]:
cc[::2]

array([    1,     4,    16,    64,   256,  1024,  4096, 16384],
      dtype=int32)